In [12]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np

DATA_PATH = "/home/giovanni/pod/thesis/code/scripts-sources/W3Pi-selection-algorithm/data/"

In [14]:
with uproot.open(DATA_PATH + "l1Nano_WTo3Pion_PU200.root") as f:
    tree = f.get("Events")
    print(tree.keys())

    branches = tree.arrays()
    genPi_etas = branches["GenPi_eta"].to_numpy()
    genPi_pts = branches["GenPi_pt"].to_numpy()

    n_gen_acceptance = 0

    for genPi_triplet_etas, genPi_triplet_pts in zip(genPi_etas, genPi_pts):
        if np.all(genPi_triplet_etas < 2.4) and np.all(genPi_triplet_pts >= 2):
            n_gen_acceptance += 1

    print(n_gen_acceptance)

['run', 'luminosityBlock', 'event', 'bunchCrossing', 'orbitNumber', 'nEle', 'Ele_hwEta', 'Ele_hwPhi', 'Ele_hwPt', 'Ele_hwQual', 'Ele_quality', 'Ele_eta', 'Ele_idScore', 'Ele_mass', 'Ele_phi', 'Ele_pt', 'Ele_puppiIsol', 'Ele_puppiIsolPV', 'Ele_trkIsol', 'Ele_trkIsolPV', 'Ele_z0', 'nGenEle', 'GenEle_isFromTau', 'GenEle_isPrompt', 'GenEle_motherId', 'GenEle_dxy', 'GenEle_eta', 'GenEle_mass', 'GenEle_phi', 'GenEle_pt', 'GenEle_z0', 'nGenJet', 'GenJet_eta', 'GenJet_mass', 'GenJet_phi', 'GenJet_pt', 'nGenPho', 'GenPho_isFromTau', 'GenPho_isPrompt', 'GenPho_motherId', 'GenPho_dxy', 'GenPho_eta', 'GenPho_mass', 'GenPho_phi', 'GenPho_pt', 'GenPho_z0', 'nGenPi', 'GenPi_charge', 'GenPi_pdgId', 'GenPi_prompt', 'GenPi_eta', 'GenPi_mass', 'GenPi_phi', 'GenPi_pt', 'GenPi_z0', 'GenVtx_x', 'GenVtx_y', 'GenVtx_z', 'nGenW', 'GenW_charge', 'GenW_pdgId', 'GenW_eta', 'GenW_mass', 'GenW_phi', 'GenW_pt', 'GenW_z0', 'nL1Vtx', 'L1Vtx_sumpt', 'L1Vtx_z', 'nPho', 'Pho_hwEta', 'Pho_hwPhi', 'Pho_hwPt', 'Pho_hwQual',

In [16]:
import h5py

DATA_PATH = "/home/giovanni/pod/thesis/code/scripts-sources/W3Pi-selection-algorithm/data/"
file = "PuppiSignal_104.hdf5"

with h5py.File(DATA_PATH + file, "r") as f:
    event = f[str(1)][()]

    c = {"pdg_id": 0, "phi": 1, "eta": 2, "pt": 3}
    ev_size = f.attrs["ev_size"]

    pdg_ids      = event[:,c["pdg_id"]].astype("int32")
    phis         = event[:,c["phi"]].astype("int32")
    etas         = event[:,c["eta"]].astype("int32")
    pts          = event[:,c["pt"]].astype("int32")

    print(etas)

[-242 -531  405 -153  523   87   -5  278  243    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
